In [25]:
from __future__ import print_function
import datetime
import logging
import time
import sys
import json
import pickle
import requests
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import google.generativeai as genai
import os
from dotenv import load_dotenv


In [23]:
load_dotenv()

True

In [5]:
credentials_filename = os.getenv("CREDENTIALS_FILENAME")

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly', 'https://www.googleapis.com/auth/tasks.readonly']

In [19]:
# get refresh token

def get_refresh_token():
    flow = InstalledAppFlow.from_client_secrets_file(
        credentials_filename,  # The downloaded JSON from Google Cloud
        SCOPES
    )
    creds = flow.run_local_server(port=8080, access_type='offline', prompt='consent')

    print("Access Token:", creds.token)
    print("Refresh Token:", creds.refresh_token)

get_refresh_token()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=724512296135-tj23hoc02avrq39dv0muap45g4eoi2dg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Ftasks.readonly&state=qfBrXCQDSoABl3nSzsNr90hCPuop2t&access_type=offline&prompt=consent
Access Token: ya29.a0AXeO80S-7oljUddE5F5Qd9Jco5iR3fvVtdkfctc8JAmNOSLpUQxsAU_IZh5ibnjg-GQ-M3KRRt0Dd7TMlskc9y2PvdAscOsyf3yfEJQZy8bYzgK55uTuRPmEounKDAtpaP4rdVb-eyZuOuoyMeq4yx1J9uJi8Hvu3CP4s5lAaCgYKAdYSARMSFQHGX2Mi_xuVO0LpSlISYIaE-T_Z1Q0175
Refresh Token: 1//09KNybr9k3kbaCgYIARAAGAkSNgF-L9Irt58X2LjtpZKOs5bzOWbhXVfporXNohuMKcoSLjD33lSnRI-z3DdV8L2vB7WqZ7Dk7A


In [6]:
def getCredsWithFile(credentials_filename):
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_filename, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

In [20]:
def getCreds():
    # Load credentials from environment variables
    client_id = os.environ.get('GOOGLE_CLIENT_ID')
    client_secret = os.environ.get('GOOGLE_CLIENT_SECRET')
    refresh_token = os.environ.get('GOOGLE_REFRESH_TOKEN')

    if not all([client_id, client_secret, refresh_token]):
        raise ValueError("Missing Google API credentials in environment variables")

    creds_data = {
        "token": None,
        "refresh_token": refresh_token,
        "token_uri": "https://oauth2.googleapis.com/token",
        "client_id": client_id,
        "client_secret": client_secret,
        "scopes": SCOPES,
    }

    creds = Credentials.from_authorized_user_info(info=creds_data)

    if not creds.valid:
        if creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            raise ValueError("Unable to refresh credentials")

    return creds

In [7]:
def connect_to_cal(creds):
    service = build('calendar', 'v3', credentials=creds)
    return service

In [8]:
def connect_to_tasks(creds):
    service = build('tasks', 'v1', credentials=creds)
    return service

In [ ]:
# get credentials
creds = getCreds()

# query calendar
service_cal = connect_to_cal(creds)
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
now_plus = (datetime.datetime.utcnow() + datetime.timedelta(days=20)).isoformat() + 'Z' # 'Z' indicates UTC time
events = service_cal.events().list(calendarId=os.getenv("CALENDAR_ID"), timeMin=now, timeMax=now_plus,
                                                  singleEvents=True,
                                                  orderBy='startTime').execute().get('items', [])

# query tasks
service_tasks = connect_to_tasks(creds)
tasklists = service_tasks.tasklists().list().execute()
items = tasklists.get('items', [])
tasks_response = service_tasks.tasks().list(tasklist=os.getenv("TASKLIST")).execute()
tasks = tasks_response.get('items', [])

C:\Users\alexa\AppData\Local\Temp\ipykernel_948\1048224468.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
C:\Users\alexa\AppData\Local\Temp\ipykernel_948\1048224468.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now_plus = (datetime.datetime.utcnow() + datetime.timedelta(days=20)).isoformat() + 'Z' # 'Z' indicates UTC time


In [33]:
GeminiApiKey = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GeminiApiKey)
model = genai.GenerativeModel('gemini-1.5-flash')

In [34]:
today_str = datetime.date.today().strftime("%Y-%m-%d")

promt = f"""
Here are the google calendar api response and google tasks. 
Give me summary for my next 3 days including today and how I should prepare. 
Also consider tasks without a date as always relevant.
Today is {today_str} and I am living in Berlin and I am working on weekdays.
Please guess what the events are and give ideas consider weather and keep it short. 
Format nicely with line breaks after roughly 50 characters.
"""

In [32]:
response = model.generate_content(promt + str(events) + str(tasks))
print(response.text)


**Next 3 Days in Berlin (Feb 5th - Feb 7th, 2025):**

**February 5th (Today):**

*   **Tasks:**  Plan picking up the children from Alexandra, clean the washing machine (video link provided), and prepare for tomorrow's DKB interview.

*   **Preparation:** Review your  product owner experience and expectations for the DKB interview.  Expect a potentially cold day in Berlin; dress warmly.

<br>

**February 6th (Thursday):**

*   **Event:** DKB Interview (11:30 AM - 12:30 PM).

*   **Preparation:**  Arrive 15 minutes early.  Dress professionally. Review your portfolio and prepare answers about your PO experience. Check the weather and dress accordingly.


<br>

**February 7th (Friday):**

*   **Tasks:** Watch "Prodigy" if you have time. Continue working on your long-term tasks like the real estate decision and finding a new energy provider.

*   **Preparation:**  It is a work day.  No major events are scheduled. The weather in Berlin might still be cold.


<br>

**Ongoing Tasks (No Due Dat

In [3]:
response = requests.post(
  		"https://api.mailgun.net/v3/sandbox4cbef2d2ae6e49718f8e85282978fa0a.mailgun.org/messages",
  		auth=("api", os.getenv('e895355012bce4e64d9bd22af5bb4eb6-d8df908e-0e94ca66', 'e895355012bce4e64d9bd22af5bb4eb6-d8df908e-0e94ca66')),
  		data={"from": "Mailgun Sandbox <postmaster@sandbox4cbef2d2ae6e49718f8e85282978fa0a.mailgun.org>",
			"to": "Alexander Kroschwald <alexander.eck@gmail.com>",
  			"subject": "Hello Alexander Kroschwald",
  			"text": "Congratulations Alexander Kroschwald, you just sent an email with Mailgun! You are truly awesome!"})

print(response.status_code, response.text) 

200 {"id":"<20250131152828.78524ba0af795b8d@sandbox4cbef2d2ae6e49718f8e85282978fa0a.mailgun.org>","message":"Queued. Thank you."}

